# FGSM vs model-based attacks

In [ ]:
import math, os, random, json, numpy as np, pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)

TRAIN_FILE = 'PowerCons_TRAIN.tsv'
TEST_FILE  = 'PowerCons_TEST.tsv'
SEQ_IN = 48
SEQ_OUT = 12
STEP = 1
EPS = 0.03

Device: cpu


In [ ]:
def load_tsv(path):
    return pd.read_csv(path, sep='\t', header=None).values.astype('float32')

raw_train_full = load_tsv(TRAIN_FILE)
raw_test  = load_tsv(TEST_FILE)
print('Raw shapes :', raw_train_full.shape, raw_test.shape)

# Нужна нормализация?
mu = raw_train_full.mean(axis=1, keepdims=True)
std = raw_train_full.std(axis=1, keepdims=True) + 1e-8
raw_train_full = (raw_train_full - mu) / std
raw_test = (raw_test - mu) / std

VAL_FRAC = 0.1
split = int(raw_train_full.shape[0] * (1 - VAL_FRAC))
train_raw, val_raw = raw_train_full[:split], raw_train_full[split:]
print('Split shapes:', train_raw.shape, val_raw.shape)

Raw shapes : (180, 145) (180, 145)
Split shapes: (162, 145) (18, 145)


In [ ]:
def create_windows(arr, seq_in=SEQ_IN, seq_out=SEQ_OUT, step=STEP, thr=0.0):
    """Преобразует массивы в выборку для бинарной классификации.
    y[t] = 1 если будущее значение > thr, иначе 0.
    Возвращает TensorDataset(X, y)."""
    X, y = [], []
    for series in arr:
        for i in range(0, len(series)-seq_in-seq_out+1, step):
            X.append(series[i:i+seq_in, None])
            target_slice = series[i+seq_in:i+seq_in+seq_out]
            y.append( (target_slice > thr).astype('float32') )
    X = torch.tensor(np.stack(X), dtype=torch.float32)
    y = torch.tensor(np.stack(y), dtype=torch.float32)
    return TensorDataset(X, y)

train_ds = create_windows(train_raw)
val_ds   = create_windows(val_raw)
test_ds  = create_windows(raw_test)

train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=128)
test_loader  = DataLoader(test_ds,  batch_size=128)

In [ ]:
class Activation(nn.Module):
    def __init__(self, act='identity'):
        super().__init__()
        if act == 'sigmoid':
            self.act = nn.Sigmoid()
        elif act == 'tanh':
            self.act = nn.Tanh()
        else:
            self.act = nn.Identity()
    def forward(self, x): return self.act(x)

class LSTM(nn.Module):

    def __init__(self, hidden_dim=64, n_layers=2, x_dim=1, output_dim=SEQ_OUT, dropout=0.2, activation_type='identity'):
        super().__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.rnn = nn.LSTM(x_dim, hidden_dim, num_layers=n_layers, dropout=dropout if n_layers>1 else 0.0, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim * n_layers, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.final_activation = Activation(activation_type)

    def forward(self, data):
        _, (hidden, _) = self.rnn(data)
        hidden = hidden.transpose(0,1).reshape(data.size(0), -1)
        hidden = self.dropout(hidden)
        out = self.relu(self.fc1(hidden))
        out = self.fc2(self.dropout(out))
        return self.final_activation(out)

model = LSTM().to(device)

In [ ]:
def evaluate(model, loader):
    model.eval()
    loss_sum, n = 0., 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            loss = F.binary_cross_entropy_with_logits(model(x), y)
            loss_sum += loss.item()*x.size(0)
            n += x.size(0)
    return loss_sum / n

opt = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.5, patience=3)

best, patience, wait = 1e9, 6, 0
for epoch in range(1, 51):
    model.train()
    run_loss, n = 0., 0
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        opt.zero_grad()
        loss = F.binary_cross_entropy_with_logits(model(x), y)
        loss.backward()
        opt.step()
        run_loss += loss.item()*x.size(0)
        n += x.size(0)
    val_loss = F.binary_cross_entropy_with_logits(model(x), y)
    scheduler.step(val_loss)
    print(f'E{epoch:02d} Train {run_loss/n:.4f} | Val {val_loss:.4f}')
    if val_loss < best-1e-4:
        best, wait = val_loss, 0
        torch.save(model.state_dict(), 'victim_best.pth')
    else:
        wait += 1
        if wait >= patience:
            print('Early stopping')
            break
model.load_state_dict(torch.load('victim_best.pth'))

E01 Train 0.6025 | Val 0.5322
E02 Train 0.5376 | Val 0.5396
E03 Train 0.5229 | Val 0.4966
E04 Train 0.5165 | Val 0.5086
E05 Train 0.5093 | Val 0.4755
E06 Train 0.5073 | Val 0.4864
E07 Train 0.5092 | Val 0.5703
E08 Train 0.5062 | Val 0.4547
E09 Train 0.5059 | Val 0.5173
E10 Train 0.4996 | Val 0.4836
E11 Train 0.4923 | Val 0.5386
E12 Train 0.4884 | Val 0.4323
E13 Train 0.4830 | Val 0.4933
E14 Train 0.4771 | Val 0.5028
E15 Train 0.4702 | Val 0.4435
E16 Train 0.4694 | Val 0.4481
E17 Train 0.4587 | Val 0.4387
E18 Train 0.4532 | Val 0.4136
E19 Train 0.4514 | Val 0.3658
E20 Train 0.4455 | Val 0.3892
E21 Train 0.4423 | Val 0.3957
E22 Train 0.4432 | Val 0.5196
E23 Train 0.4365 | Val 0.4355
E24 Train 0.4297 | Val 0.4272
E25 Train 0.4240 | Val 0.4562
Early stopping


<All keys matched successfully>

In [ ]:
class SurrogateNet(nn.Module):

    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv1d(1, 32, 3, padding=1), nn.ReLU(),
            nn.Conv1d(32, 64, 3, padding=1), nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64*SEQ_IN, 256), nn.ReLU(),
            nn.Linear(256, SEQ_IN),
            nn.Tanh())

    def forward(self, x):
        x = x.transpose(1,2)
        out = self.net(x)
        return out.unsqueeze(-1)

surrogate = SurrogateNet().to(device)

In [ ]:
for p in model.parameters():
    p.requires_grad_(False)
model.eval()

def train_surrogate(surr, victim, loader, eps=EPS, epochs=15, lr=1e-4, alpha_l2=1e-3):
    opt = torch.optim.Adam(surr.parameters(), lr)
    for ep in range(1, epochs+1):
        surr.train(); run_vloss = 0.
        eps_curr = eps * (ep/epochs)
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            delta = eps_curr * torch.tanh(surr(x))
            x_adv = torch.clamp(x + delta, -1, 1)
            vloss = F.binary_cross_entropy_with_logits(victim(x_adv), y)
            reg = alpha_l2 * (delta**2).mean()
            loss = -(vloss - reg)
            opt.zero_grad()
            loss.backward()
            opt.step()
            run_vloss += vloss.item()*x.size(0)
        print(f'Surr E{ep:02d} victim‑loss {run_vloss/len(loader.dataset):.4f}')
    torch.save(surr.state_dict(), 'surrogate_maxloss.pth')

train_surrogate(surrogate, model, train_loader)

Surr E01 victim‑loss 0.4903
Surr E02 victim‑loss 0.4910
Surr E03 victim‑loss 0.4919
Surr E04 victim‑loss 0.4928
Surr E05 victim‑loss 0.4937
Surr E06 victim‑loss 0.4947
Surr E07 victim‑loss 0.4958
Surr E08 victim‑loss 0.4970
Surr E09 victim‑loss 0.4981
Surr E10 victim‑loss 0.4992
Surr E11 victim‑loss 0.5007
Surr E12 victim‑loss 0.5019
Surr E13 victim‑loss 0.5034
Surr E14 victim‑loss 0.5048
Surr E15 victim‑loss 0.5062


In [ ]:
class Attack:
    def __init__(self, eps, clamp=(-1,1)):
        self.eps = eps
        self.clamp = clamp

class FGSMAttack(Attack):
    def __call__(self, model, x, y):
        x_req = x.clone().detach().requires_grad_(True)
        loss = F.binary_cross_entropy_with_logits(model(x_req), y)
        loss.backward()
        delta = self.eps * x_req.grad.sign()
        x_adv = torch.clamp(x + delta, *self.clamp)
        return x_adv.detach()

class ModelBasedAttack(Attack):

    def __init__(self, surrogate, eps, clamp=(-1,1)):
        super().__init__(eps, clamp)
        self.surr = surrogate.eval()

    @torch.no_grad()
    def __call__(self, model, x, y):
        delta = self.eps * torch.tanh(self.surr(x))
        return torch.clamp(x + delta, *self.clamp)

fgsm_attack  = FGSMAttack(EPS)
model_attack = ModelBasedAttack(surrogate, EPS)

In [ ]:
def preds_to_labels(logits):
    return (torch.sigmoid(logits) >= 0.5).float()

def fooling_rate(model, loader, attack):
    model.eval()
    fooled, total = 0, 0
    for x, y in loader:
        x, y = x.to(device), y.to(device)
        preds_orig = preds_to_labels(model(x).detach())
        x_adv = attack(model, x, y)
        preds_adv = preds_to_labels(model(x_adv).detach())
        batch_fooled = (preds_adv != preds_orig).any(dim=1)
        fooled += batch_fooled.sum().item()
        total += x.size(0)
    return fooled / total

Fooling rate FGSM  : 0.514
Fooling rate Model : 0.476
